In [1]:
from __future__ import annotations

import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

def pick_existing(paths: list[Path]) -> Path:
    for p in paths:
        if p.exists():
            return p
    raise FileNotFoundError("No candidate path exists\n" + "\n".join(map(str, paths)))

CANDIDATE_MA_ROOTS = [
    Path("/scion/5261/econ470001/ma-data/ma"),
    Path("/econ470/a0/work/ma-data/ma"),
    Path.cwd().parent / "ma-data" / "ma",
]

MA_ROOT = pick_existing(CANDIDATE_MA_ROOTS)
LANDSCAPE_DIR = pick_existing([MA_ROOT / "landscape" / "Extracted Data"])
FFSCOST_ROOT = pick_existing([MA_ROOT.parent / "ffs-costs"])
PEN_DIR = pick_existing([MA_ROOT / "penetration" / "Extracted Data"])

# Standardize: write build outputs to processed for analysis
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

YEARS = list(range(2014, 2020))

print("MA_ROOT", MA_ROOT)
print("LANDSCAPE_DIR", LANDSCAPE_DIR)
print("FFSCOST_ROOT", FFSCOST_ROOT)
print("PEN_DIR", PEN_DIR)
print("OUT_DIR", OUT_DIR.resolve())
print("Target years", YEARS)

MA_ROOT /scion/5261/econ470001/ma-data/ma
LANDSCAPE_DIR /scion/5261/econ470001/ma-data/ma/landscape/Extracted Data
FFSCOST_ROOT /scion/5261/econ470001/ma-data/ffs-costs
PEN_DIR /scion/5261/econ470001/ma-data/ma/penetration/Extracted Data
OUT_DIR /home/rpat638/econ470/a0/work/hwk2/data/processed
Target years [2014, 2015, 2016, 2017, 2018, 2019]


In [2]:
def norm_colname(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"\s+", "_", s)
    s = re.sub(r"[^a-z0-9_]", "", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def parse_year_from_name(p: Path) -> int | None:
    m = re.search(r"(20\d{2})", p.name)
    return int(m.group(1)) if m else None

def detect_header_row_csv(path: Path, nrows: int = 100) -> int:
    preview = pd.read_csv(
        path,
        header=None,
        nrows=nrows,
        dtype=str,
        encoding_errors="replace",
        engine="python",
        on_bad_lines="skip",
    ).fillna("").astype(str)

    keys = [
        "state", "county", "contract", "contract id",
        "plan", "plan id", "pbp",
        "organization", "bid", "benchmark", "premium",
        "code", "enrollment", "reimbursement", "per capita",
        "fips", "ssa", "penetration", "eligibles", "enrolled",
    ]

    best_i = 0
    best_hits = -1
    for i in range(preview.shape[0]):
        row = " ".join(preview.iloc[i].tolist()).lower()
        hits = sum(1 for k in keys if k in row)
        if hits > best_hits:
            best_hits = hits
            best_i = i
    return best_i

def read_csv_autoheader(path: Path) -> pd.DataFrame:
    header_row = detect_header_row_csv(path)
    df = pd.read_csv(
        path,
        skiprows=header_row,
        header=0,
        dtype=str,
        encoding_errors="replace",
        engine="python",
        on_bad_lines="skip",
    )
    df.columns = [norm_colname(c) for c in df.columns]
    df = df.dropna(how="all")
    return df

def detect_header_row_excel(path: Path, nrows: int = 80) -> int:
    preview = pd.read_excel(path, header=None, nrows=nrows, engine="openpyxl").fillna("").astype(str)
    keys = ["code", "state", "county", "part a", "part b", "enrollment", "reimbursement", "per capita"]
    best_i = 0
    best_hits = -1
    for i in range(preview.shape[0]):
        row = " ".join(preview.iloc[i].tolist()).lower()
        hits = sum(1 for k in keys if k in row)
        if hits > best_hits:
            best_hits = hits
            best_i = i
    return best_i

def read_excel_autoheader(path: Path) -> pd.DataFrame:
    header_row = detect_header_row_excel(path)
    df = pd.read_excel(path, skiprows=header_row, header=0, engine="openpyxl", dtype=str)
    df.columns = [norm_colname(c) for c in df.columns]
    df = df.dropna(how="all")
    return df

def first_col(cols: list[str], needles: list[str]) -> str | None:
    cols_l = [c.lower() for c in cols]
    for n in needles:
        for i, c in enumerate(cols_l):
            if n in c:
                return cols[i]
    return None

def digits5(s: pd.Series) -> pd.Series:
    out = (
        s.astype(str).str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .str.replace(r"[^0-9]", "", regex=True)
        .str.zfill(5)
    )
    out = out.where(out.str.len() == 5, np.nan)
    return out

def digits2(s: pd.Series) -> pd.Series:
    out = (
        s.astype(str).str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .str.replace(r"[^0-9]", "", regex=True)
        .str.zfill(2)
    )
    out = out.where(out.str.len() == 2, np.nan)
    return out

def digits3(s: pd.Series) -> pd.Series:
    out = (
        s.astype(str).str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .str.replace(r"[^0-9]", "", regex=True)
        .str.zfill(3)
    )
    out = out.where(out.str.len() == 3, np.nan)
    return out

def clean_contract_id(x: pd.Series) -> pd.Series:
    s = x.astype(str).str.strip()
    s = s.replace({"nan": np.nan, "none": np.nan, "None": np.nan, "": np.nan})
    s = s.str.replace(r"\.0$", "", regex=True)
    return s

def clean_plan_id_3(x: pd.Series) -> pd.Series:
    s = x.astype(str).str.strip()
    s = s.replace({"nan": np.nan, "none": np.nan, "None": np.nan, "": np.nan})
    s = s.str.replace(r"\.0$", "", regex=True)
    s = s.str.replace(r"[^0-9]", "", regex=True)
    s = s.where(s.str.len() > 0, np.nan)
    return s.str.zfill(3)

def to_num(s: pd.Series) -> pd.Series:
    return pd.to_numeric(
        s.astype(str)
         .str.replace(",", "", regex=False)
         .str.replace("$", "", regex=False)
         .str.replace("%", "", regex=False)
         .str.strip(),
        errors="coerce",
    )

In [3]:
def landscape_files_for_year(root: Path, year: int) -> list[Path]:
    files = sorted([p for p in root.rglob("*.csv") if p.is_file()])
    hits = []
    for p in files:
        y = parse_year_from_name(p)
        if y != year:
            continue
        n = p.name.lower()
        if "sanction" in n or "importantnotes" in n or "partd" in n or "part_d" in n or "premium" in n:
            continue
        hits.append(p)
    return hits

def build_landscape_clean(years: list[int]) -> pd.DataFrame:
    out = []
    for y in years:
        files = landscape_files_for_year(LANDSCAPE_DIR, y)
        if not files:
            print("No landscape CSVs found for", y)
            continue

        chunks = []
        for p in files:
            df = read_csv_autoheader(p)
            df["year"] = y
            df["source_file"] = p.name
            chunks.append(df)
            print("loaded", y, p.name, "shape", df.shape)

        d = pd.concat(chunks, ignore_index=True)
        cols = list(d.columns)

        contract_col = first_col(cols, ["contract_id", "contractid", "contract"])
        planid_col = first_col(cols, ["plan_id", "planid", "pbp"])
        if planid_col is None:
            planid_col = first_col(cols, ["plan"])
        state_col = first_col(cols, ["state"])
        county_col = first_col(cols, ["county"])
        bid_col = first_col(cols, ["bid"])
        bmk_col = first_col(cols, ["benchmark"])
        org_col = first_col(cols, ["organization", "org"])
        pname_col = first_col(cols, ["plan_name", "planname"])

        if contract_col is None or planid_col is None:
            print("Year", y, "missing contract or plan columns")
            print("First 80 columns", cols[:80])
            continue

        dd = pd.DataFrame()
        dd["contract_id"] = clean_contract_id(d[contract_col])
        dd["plan_id"] = clean_plan_id_3(d[planid_col])
        dd["state"] = d[state_col] if state_col is not None else np.nan
        dd["county"] = d[county_col] if county_col is not None else np.nan
        dd["organization_name"] = d[org_col] if org_col is not None else np.nan
        dd["plan_name"] = d[pname_col] if pname_col is not None else np.nan
        dd["bid"] = to_num(d[bid_col]) if bid_col is not None else np.nan
        dd["benchmark"] = to_num(d[bmk_col]) if bmk_col is not None else np.nan
        dd["year"] = y
        dd["source_file"] = d["source_file"]

        dd = dd.dropna(subset=["contract_id", "plan_id"]).copy()
        out.append(dd)

    if not out:
        raise RuntimeError("No landscape data loaded")
    return pd.concat(out, ignore_index=True)

landscape_clean = build_landscape_clean(YEARS)
print("landscape_clean shape", landscape_clean.shape)

landscape_out = OUT_DIR / "landscape_clean_2014_2019.csv"
landscape_clean.to_csv(landscape_out, index=False)
print("Wrote", landscape_out)

loaded 2014 2014LandscapeSource file MA_AtoM 05292014.csv shape (15444, 17)
loaded 2014 2014LandscapeSource file MA_NtoW 05292014.csv shape (19536, 17)
loaded 2014 508_AlabamatoMontana 05292014.csv shape (15855, 29)
loaded 2014 508_NebraskatoWyoming 05292014.csv shape (20301, 29)
loaded 2015 2015LandscapeSource file MA_AtoM 11042014.csv shape (15881, 17)
loaded 2015 2015LandscapeSource file MA_NtoW 11042014.csv shape (17695, 17)
loaded 2015 508_AlabamatoMontana 03182015.csv shape (16662, 28)
loaded 2015 508_NebraskatoWyoming 03182015.csv shape (5675, 28)
loaded 2016 2016LandscapeSource file MA_AtoM 04222016.csv shape (16096, 18)
loaded 2016 2016LandscapeSource file MA_NtoW 04222016.csv shape (18122, 18)
loaded 2016 508_AlabamatoMontana 04222016.csv shape (17046, 28)
loaded 2016 508_NebraskatoWyoming 04222016.csv shape (20396, 28)
loaded 2016 508_AlabamatoMontana 10182016.csv shape (17653, 30)
loaded 2016 508_NebraskatoWyoming 10182016.csv shape (20213, 28)
loaded 2017 2017LandscapeSour

In [4]:
try:
    import openpyxl  # noqa
    print("openpyxl available")
except Exception:
    print("openpyxl missing, installing")
    !{sys.executable} -m pip install --user openpyxl

def penetration_files_for_year(year: int) -> list[Path]:
    files = [p for p in PEN_DIR.rglob("*") if p.is_file() and p.suffix.lower() in [".csv", ".txt"]]
    hits = [p for p in files if re.search(rf"{year}", p.name)]
    hits = sorted(hits, key=lambda p: (len(p.name), p.name.lower()))
    return hits

def load_penetration_crosswalk(year: int) -> pd.DataFrame:
    cands = penetration_files_for_year(year)
    if not cands:
        raise FileNotFoundError(f"No penetration files found for {year} under {PEN_DIR}")

    pick = None
    for p in cands:
        if re.search(r"state_county_penetration", p.name.lower()):
            pick = p
            break
    if pick is None:
        pick = cands[0]

    df = read_csv_autoheader(pick)
    cols = list(df.columns)

    # county fips
    if "fips" in df.columns:
        fips5 = digits5(df["fips"])
    else:
        fipsst = first_col(cols, ["fipsst"])
        fipscnty = first_col(cols, ["fipscnty"])
        if fipsst is None or fipscnty is None:
            raise KeyError(f"Penetration file missing fips columns: {pick.name}")
        fips5 = digits2(df[fipsst]) + digits3(df[fipscnty])

    # ssa code
    if "ssa" in df.columns:
        ssa5 = digits5(df["ssa"])
    else:
        ssast = first_col(cols, ["ssast"])
        ssacnty = first_col(cols, ["ssacnty"])
        if ssast is None or ssacnty is None:
            raise KeyError(f"Penetration file missing ssa columns: {pick.name}")
        ssa5 = digits2(df[ssast]) + digits3(df[ssacnty])

    out = pd.DataFrame({"year": int(year), "ssa": ssa5, "fips": fips5})
    out = out.dropna(subset=["ssa", "fips"]).drop_duplicates(subset=["year", "ssa", "fips"]).copy()

    # drop junk totals if present
    out = out[(out["ssa"] != "00000") & (out["fips"] != "00000")].copy()

    print("Pen crosswalk", year, "rows", len(out), "from", pick.name)
    return out

pen_cw_all = pd.concat([load_penetration_crosswalk(y) for y in YEARS], ignore_index=True)
print("pen_cw_all shape", pen_cw_all.shape)

openpyxl available
Pen crosswalk 2014 rows 3222 from State_County_Penetration_MA_2014_01.csv
Pen crosswalk 2015 rows 3222 from State_County_Penetration_MA_2015_01.csv
Pen crosswalk 2016 rows 3141 from State_County_Penetration_MA_2016_01.csv
Pen crosswalk 2017 rows 3219 from State_County_Penetration_MA_2017_01.csv
Pen crosswalk 2018 rows 3219 from State_County_Penetration_MA_2018_01.csv
Pen crosswalk 2019 rows 3215 from State_County_Penetration_MA_2019_01.csv
pen_cw_all shape (19238, 3)


In [5]:
MASTER_PATH = FFSCOST_ROOT / "CMS FFS Costs.csv"
print("MASTER_PATH exists", MASTER_PATH.exists(), MASTER_PATH)

def compute_ffs_cost(df: pd.DataFrame) -> pd.Series:
    cols = list(df.columns)

    a_en_col = first_col(cols, ["part_a_enrollment"])
    b_en_col = first_col(cols, ["part_b_enrollment"])
    a_rb_col = first_col(cols, ["part_a_total_reimbursement"])
    b_rb_col = first_col(cols, ["part_b_total_reimbursement"])

    if a_en_col is None or b_en_col is None or a_rb_col is None or b_rb_col is None:
        raise KeyError("FFS file missing A/B enrollment or reimbursement columns")

    denom = (to_num(df[a_en_col]) + to_num(df[b_en_col])).replace(0, np.nan)
    return (to_num(df[a_rb_col]) + to_num(df[b_rb_col])) / denom

def load_ffs_from_master(master_path: Path) -> pd.DataFrame:
    df = read_csv_autoheader(master_path)
    cols = list(df.columns)

    code_col = first_col(cols, ["code"])
    year_col = first_col(cols, ["year"])

    if code_col is None or year_col is None:
        raise KeyError("FFS master missing code or year")

    tmp = df.copy()
    tmp["year"] = pd.to_numeric(tmp[year_col], errors="coerce").astype("Int64")
    tmp["ssa"] = digits5(tmp[code_col])
    tmp = tmp.dropna(subset=["year", "ssa"]).copy()
    tmp["year"] = tmp["year"].astype(int)

    out_all = []
    for y in sorted(tmp["year"].unique().tolist()):
        if y not in YEARS:
            continue
        cw = pen_cw_all[pen_cw_all["year"] == y][["ssa", "fips"]].copy()
        m = tmp[tmp["year"] == y].merge(cw, on="ssa", how="left")
        m["ffs_cost"] = compute_ffs_cost(m)
        out = m[["fips", "year", "ffs_cost"]].dropna(subset=["fips", "ffs_cost"]).copy()
        out_all.append(out)

    out = pd.concat(out_all, ignore_index=True).drop_duplicates(subset=["fips", "year"])
    return out

def find_ffs_excel(ffscost_root: Path, year: int) -> Path | None:
    extracted = ffscost_root / "Extracted Data"
    yy = str(year)[-2:]
    pats = [f"*FFS{yy}*.xlsx", f"*ffs{yy}*.xlsx", f"*FFS{year}*.xlsx", f"*ffs{year}*.xlsx"]
    hits = []
    for pat in pats:
        hits.extend(list(extracted.rglob(pat)))
    hits = [p for p in hits if p.is_file() and "~$" not in p.name]
    if not hits:
        return None
    hits = sorted(hits, key=lambda p: (len(str(p)), str(p).lower()))
    return hits[0]

def load_ffs_from_excel_year(ffscost_root: Path, year: int) -> pd.DataFrame:
    xlsx_path = find_ffs_excel(ffscost_root, year)
    if xlsx_path is None:
        raise FileNotFoundError(f"Missing Excel for {year} under {ffscost_root}/Extracted Data")

    df = read_excel_autoheader(xlsx_path)
    cols = list(df.columns)

    code_col = first_col(cols, ["code"])
    if code_col is None:
        raise KeyError(f"Excel missing code column in {xlsx_path.name}")

    tmp = df.copy()
    tmp["ssa"] = digits5(tmp[code_col])
    tmp["year"] = int(year)

    cw = pen_cw_all[pen_cw_all["year"] == year][["ssa", "fips"]].copy()
    m = tmp.merge(cw, on="ssa", how="left")

    m["ffs_cost"] = compute_ffs_cost(m)
    out = m[["fips", "year", "ffs_cost"]].dropna(subset=["fips", "ffs_cost"]).copy()

    print("Loaded", year, "from", xlsx_path, "rows", len(out))
    return out

pieces = []
if MASTER_PATH.exists():
    pieces.append(load_ffs_from_master(MASTER_PATH))

for y in [2016, 2017, 2018, 2019]:
    pieces.append(load_ffs_from_excel_year(FFSCOST_ROOT, y))

ffs_cost = pd.concat(pieces, ignore_index=True).drop_duplicates(subset=["fips", "year"]).copy()
ffs_cost = ffs_cost[ffs_cost["year"].isin(YEARS)].copy()

ffs_out = OUT_DIR / "ffs_cost_2014_2019.csv"
ffs_cost.to_csv(ffs_out, index=False)
print("Wrote", ffs_out, "rows", len(ffs_cost))

MASTER_PATH exists True /scion/5261/econ470001/ma-data/ffs-costs/CMS FFS Costs.csv
Loaded 2016 from /scion/5261/econ470001/ma-data/ffs-costs/Extracted Data/FFS16.xlsx rows 3126
Loaded 2017 from /scion/5261/econ470001/ma-data/ffs-costs/Extracted Data/ffs2017/FFS17.xlsx rows 3205
Loaded 2018 from /scion/5261/econ470001/ma-data/ffs-costs/Extracted Data/FFS2018/FFS18.xlsx rows 3205
Loaded 2019 from /scion/5261/econ470001/ma-data/ffs-costs/Extracted Data/FFS2019/FFS19.xlsx rows 3203
Wrote data/processed/ffs_cost_2014_2019.csv rows 19147


In [6]:
def quartile_codes(x: pd.Series) -> pd.Series:
    x = pd.to_numeric(x, errors="coerce").replace([np.inf, -np.inf], np.nan)
    cats = pd.qcut(x, 4, duplicates="drop")
    codes = cats.cat.codes.replace(-1, np.nan) + 1
    return codes.astype("Int64")

ffs18 = ffs_cost[ffs_cost["year"] == 2018].copy()
ffs18["ffs_quartile"] = quartile_codes(ffs18["ffs_cost"])

ffs18_out = OUT_DIR / "ffs_cost_2018_with_quartiles.csv"
ffs18[["fips", "year", "ffs_cost", "ffs_quartile"]].to_csv(ffs18_out, index=False)
print("Wrote", ffs18_out, "rows", len(ffs18))

# Optional: quick merge check if you already built county_bid_hhi_2018
county_path = OUT_DIR / "county_bid_hhi_2018.csv"
if county_path.exists():
    county = pd.read_csv(county_path, dtype={"fips": str}, low_memory=False)
    county["fips"] = county["fips"].astype(str).str.zfill(5)
    county["year"] = pd.to_numeric(county["year"], errors="coerce").fillna(2018).astype(int)

    test = county.merge(ffs18[["fips", "year", "ffs_quartile"]], on=["fips", "year"], how="left")
    print("Post merge quartile missing rate:", test["ffs_quartile"].isna().mean())
    print("Merged rows:", test.shape)
else:
    print("Skipping county merge check; missing", county_path)

Wrote data/processed/ffs_cost_2018_with_quartiles.csv rows 3205
Post merge quartile missing rate: 0.006821705426356589
Merged rows: (3225, 10)
